In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime as dt 

In [0]:
df = spark.table('default.sample_superstore').toPandas()

qnt_linhas = str(df.shape)
nome_colunas = str(df.columns.tolist())
tipo_dados = str(df.dtypes.to_dict())
estat_resumo = df['Sales'].describe().to_string() # valido somente para variaveis quantitativa
qnt_duplicados = str(df[df.duplicated()].shape[0])
qnt_nulos = str(df.isnull().sum().to_dict())  

analise_exploratoria = pd.DataFrame({
    'Análise': ['Quantidade linhas/colunas', 'Nome das colunas', 'Tipo de dados das colunas', 
        'Resumo estatístico (Vendas)', 'Total dados duplicados', 'Total dados nulos'],
    'Resultados': [qnt_linhas, nome_colunas, tipo_dados, estat_resumo, qnt_duplicados, qnt_nulos]
})

display(analise_exploratoria)
analise_exploratoria.to_markdown()

 **1** - Qual a **cidade** com maior **valor de venda** de produtos da **categoria** Office Supplies?

In [0]:
filtro_categoria = df[df['Category'] == 'Office Supplies']
group_cidade_venda = filtro_categoria.groupby('City', as_index= False)['Sales'].sum()
ranking = group_cidade_venda.sort_values('Sales', ascending = False).head(3)
display(ranking)
# ranking.to_markdown()

**2** - Qual o total de **vendas** por **data do pedido**? (Em gráfico de barras)

In [0]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
group_vendas_data = df.groupby('Order Date')['Sales'].sum().reset_index()
plt.figure(figsize=(20, 6))
plt.plot(group_vendas_data['Order Date'], group_vendas_data['Sales'], color='green')
plt.title('Total de vendas por data de pedido')
plt.xlabel('Order Date')
plt.ylabel('Sales')
# plt.savefig('total_vendas_data.png')
plt.show()

**3** - Qual o total de **vendas** por **estado**? (Em gráfico de barras)

In [0]:
group_estado_venda = df.groupby('State')['Sales'].sum().reset_index()
plt.figure(figsize = (16, 6))
sns.barplot(data = group_estado_venda, y = 'Sales', x = 'State').set(title = 'Quantidade de vendas por estado')
plt.xticks(rotation = 80)
# plt.savefig('quantidade_vendas_estado.png')
plt.show()

**4** - Quais são as **10 cidades** com mais **vendas**? (Em gráfico de barras)

In [0]:
group_vendas_cidades = df.groupby('City')['Sales'].sum().reset_index()
top_vendas_cidades = group_vendas_cidades.sort_values(by = 'Sales', ascending = False).head(10)
plt.figure(figsize = (16, 6))
sns.set_palette('coolwarm')
sns.barplot(data = top_vendas_cidades, x = 'Sales', y = 'City').set(title = 'Top 10 cidades com mais vendas')
plt.show()

**5** - Qual o **segmento** com mais **vendas**? (Em gráfico de pizza)

In [0]:
df['Sales'] = df['Sales'].astype(float)
group_segmentos_vendas = df.groupby('Segment')['Sales'].sum().reset_index()
top_segmentos_vendas = group_segmentos_vendas.sort_values(by='Sales', ascending=False)

def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return ' $ {v:d}'.format(v=val)
    return my_format

plt.figure(figsize=(16, 6))
plt.pie(top_segmentos_vendas['Sales'], labels=top_segmentos_vendas['Segment'], autopct=autopct_format(top_segmentos_vendas['Sales'].values), startangle=90)
centro_circulo = plt.Circle((0, 0), 0.82, fc='white')
figura = plt.gcf()
figura.gca().add_artist(centro_circulo)

plt.annotate(text='Total vendas: ' + ' $ ' + str(int(sum(top_segmentos_vendas['Sales']))), xy=(-0.25, 0))
plt.title('Total de vendas por Segmento')
plt.show()